# Assembly Lab

- Authors:
  - Quinn Ha, hab8@mcmaster.ca
  - Fondson Lu, luh57@mcmaster.ca
- Group ID on Avenue: 58
- Gitlab URL: https://gitlab.cas.mcmaster.ca/hab8/l3-assembly

## F1

### Python -> Pep/9 Translation
simple.py
```
     BR     program
x:   .BLOCK 2
program:    LDWA 3,i
            ADDA 2,i
            STWA x,d
            DECO x,d
            .END
```

add_sub.py
```
        BR       program
value:  .BLOCK 6
result:  .BLOCK 6
_UNIV:   .WORD 42
variable:        .WORD -3

program: DECI    value,d
         LDWA    _UNIV,d
         ADDA    value,d
         ADDA    variable,d
         ADDA    -1,i
         STWA    result,d
         DECO    result,d
         .END
```

### Why Does the Translator use NOP1 Instructions?
The purpose of NOP1 is to prevent hazards by rendering previous instructions void. The translator uses it to ensure a clean working environment for machine execution. In addition, we cannot guarantee that an instruction will follow after a label, so adding a NOP1 statement ensures that there is a non-obtrusive instruction for the label.

### Visitor
The purpose of the visitor objects is to recusively traverse the syntax tree to read and translate the python code into abstract syntax, utilizing the ast module to inherit behaviour from NodeVisitor class.

### Generator
The purpose of the generator objects is to generate assembly code based off some data, whether it be results from a visitor, or other data. It can also be used for cross-compatible code to be generated given a set of information!

### Limitations
Firstly, the current code is very simple, and does not handle cases such as control flows, functions, and variable scope definitions. The current code is also platform specific and is not extendable for other platforms, violating the open-closed principle. In addition, the the code is not scalable, once again violating the open-closed principle if more instructions need to be added, or further optimizations need to be implemented.

## F2

### Improvements

To visit each node, code was edited in the Global Variable visitor, as the translator first initializes all the variables from that visitor before calling the TopLevelProgram to parse the rest. This allowed us to get the relevant information regarding each variable, including information such as the type of assignment, name, and value. In addition, some changes needed to be made in our Static Memory Allocation generator to create the respective optimizations whether it be better memory allocation, constants, or changes based off the symbol table. The following optimizations are made based off these changes.

#### Memory Allocation
To improve the memory allocation, on the first visit of the variable, we stored the value of the assignment alongside the variable name (and generated name), for which was passed in the generator to decouple into WORD, EQUATE, and BLOCK statements. When a variable was initialized with the WORD or EQUATE keyword, we would then store that variable name into a "remove list", for which the first set of LDWA and STWA instructions will then be removed from the set of instructions!

#### Constants
To improve on the constants, we stored the old variable name, and passed it to the generator, for which detected if it was a constant from the variable name. If it was a constant, the keyword EQUATE would have been selected over WORD and BLOCK. Then the same logic applies for removing the first set of LDWA and STWA instruction which corresponds to that variable.

#### Symbol table
We created a generator object which utilized a set of 8 upper case alpha numberic characters, starting at AAAAAAAA, assigning each new variable a new set of letters. These letters rotate from AAAAAAAA to AAAAAAAB to AAAAAAAC onwards, generating 26^8 combinations of variable assignments via our symbol table.

### Overflow
Overflow restricts how large of a number that our program can store, while in PEP9 we have 16 bits, we are restricted between -2^16 and 2^16 -1 sized number. If we hit the upper bound and add 1, the number would overflow, and the number 0 would be stored (0000000000000000). In various other programming languages, an error would be thrown instead, or it would be converted into a new datatype with more bits which can contain the new number (int vs long in C).

### F3
#### GCD Manual Translation
```

                BR t1
a:		.BLOCK 2   
b:		.BLOCK 2

t1:		DECI a, d
		DECI b, d
while:	BREQ end
		LDWA a, d
		SUBA b, d
if:		BRLE else
		STWA a, d
		BR while
else:		LDWA b, d
		SUBA a, d
		STWA b, d
		BR while
end:		DECO a, d
.END


```
#### Implementation
To implement these changes, a new visit function for if conditions can be made, and placed in TopLevelProgram. It will then be parsed, and recursively visit every element (statement) inside the if condition. We can then take inspiration from the code available in the visit_While function to generate the required statements.

### F4
#### Translation Complexity
From least difficult to most difficult

call_void
- Only has 1 function with no parameter and no value to return, thus the least difficult to translate.

call_param
- call_param is similar to call_void, however has a parameter, increasing the difficulty level to translate due to needing to update the relevant memory for the function to utilize.

call_return
- call_return has a parameter and a value to return, which increases the difficulty in comparison to the previous files due to needing to load the accumulator before returning in the function.

fibonnaci
- Similar to call_return with a parameter and return value, with slightly more complex code to translate due to the while loop, requireing more instructions.

factorial
- factorial uses nested functions, calling one function in another. The difficulty increases due to the need to increase and decrease the stack pointer an additional time due to the extra 
call. The implementation difficulty is only slightly more difficult, however, as correct function implementation will update the stack relative to the pointer, which ensures consistency across all levels of functions.

fac_rec
- fac_rec uses nested functions and is recursive,increasing the difficulty by recursively updating the stack due to self calls. The stack can also possibly overflow, which could be guarded agaisnt.

fib_rec
- Similar to fac_rec, however with twice the recursion calls! Hence it is the most difficult.

#### Translations
##### call_param.py
```
         BR main
x:       .BLOCK 2
_UNIV:   .EQUATE 42
         ;;;; my_func ;;;;
value:   .EQUATE 0           ;formal parameter #2d
result:  .EQUATE 2           ;local variable #2d

my_func: SUBSP 4, i          ;push #result #value
         LDWA value, s
         ADDA _UNIV, i
         STWA result, s
         DECO result, s
         ADDSP 4, i          ;pop #result #value 
         RET

main:    DECI x, d
         LDWA x, d
         STWA -6, s
         CALL my_func
         ADDSP 2, i          ;pop #value
         .END
```
##### call_return.py
```
         BR main
         ;;;; main ;;;;;
x:       .BLOCK 2
_UNIV:   .EQUATE 42          ;global variable
retVal:  .EQUATE 6           ;local variable #2d
         ;;;; my_func ;;;;
value:   .EQUATE 0           ;formal parameter #2d
result:  .EQUATE 2           ;local variable #2d

my_func: SUBSP 4, i          ;push #result #value
         LDWA value, s
         ADDA _UNIV, i
         STWA result, s
         STWA retVal,s
         ADDSP 4, i          ;pop #result #value 
         RET

main:    DECI x, d
         LDWA x, d
         SUBSP 2, i          ;push #retVal 
         STWA -6, s
         CALL my_func
         DECO 0, s
         ADDSP 2, i          ;pop #value
         .END
```
##### call_void.py
```
         BR main
         ;;;; main ;;;;;
_UNIV:   .EQUATE 42          ;global variable
         ;;;; my_func ;;;;
value:   .EQUATE 0            ;local variable #2d
result:  .EQUATE 2           ;local variable #2d

my_func: SUBSP 4, i          ;push #result #value
         DECI value, s
         LDWA value, s
         ADDA _UNIV, i
         STWA result, s
         DECO result, s
         ADDSP 4, i          ;pop #result #value
         RET

main:    CALL my_func
         .END
```

#### Implementation
The implementation can be broken down into various steps:
- Allocating memory for local functions
- Parsing local functions
    - Modifying access types (s, i, d)
    - Implementing Parameters
    - Implementing Returns

It's important to clarify how our implementation allocates memory, as it does not follow the format presented in lecture. The structure is as follows:

```
local_vars
parameters
ret_addr
global_vars
///////////
```
Having parameters stored under the return address allows for easy referencing within the function, as only the stack size is needed to calculate the location in the stack to address. The presented way functions as well, however this method proved easier for the group to implement and comprehend.

#####  Allocating Memory
To allocate memory, a new visitor/generator pair can be used, similar to how global variables are extracted. Instead of walking through the AST, each FunctionDef object contains a body with all the statements local to the function. Then we can iterate through each statement, and store the unique variables which are assign statements. In addition, If, While, and For statements need to further be parsed as various other local variables can be present, and nested.

##### Parsing local functions

##### Access Types
We can borrow the logic from TopLevelProgram to generate the relevant code, making sure to be cautious about what kind of variable is used, as it could be on the stack (s), and intermediate (i), or from a global variable (d). __access_memory function can be edited to verify what kind of variable is being accessed, and select the appropriate identifier!

##### Parameters
As parameters are stored directly under the return address, we need to first push the value on top of the stack (from the caller) with the formula -(2 + 2 * parameter_number), and then allocate the relevant space in the callee with the formula 2 * total_number_parameters. The values for the parameters will automatically be populated as a result. This allows for us to save an extra ADDSP call, since we can deallocate the parameters and local variables in one statement (in the callee).

##### Return
As we are only concerned with single value returns, it's easiest to store the result into the accumulator from the callee, and then store from the accumulator into a variable from the caller. In addition, a RET expression is generated when the NodeVisitor encounters a ast.Return typed statement.

#### Stack Overflow
Pep/9 automatically detects if the stack is too large during runtime (especially in recursive applications), and kills the running program. Other languages may throw an exception, or cause segmentations fault.

### F5

#### Global Arrays
To implement global arrays, we need to first implement visiting static array initialization, and subscripting variables to access the information inside the array (at a given index). ```.BLOCK x``` needs to be used to statically allocate the array, where x is 2 * array size (every element is 2 bits). Various other instructions also need to be implemented to support array loading and storing as well. 

We also need to check if each variable instruction may pertain to an array, which can be checked with an _ at the end of the variable name, changing the addressing mode from d to x.

#### Local Arrays
The implementation strategy is very similar to global arrays as how global variables are to local variables, which means we can reuse the code developped for global arrays. Firstly the addressing mode is now sx instead of x. Next, when allocating memory on the stack, the location will now depend on 2 * array size for each array in the stack frame.

The implementation was not completed due to time and resource contraints.

#### Unbounded Arrays
One way this could be implemented is calculating the space needed for each local/global variable, and allocating the rest of the memory to the arrays. This method is "pseudo-unbounded" where it may seem that the array has infinite space, however is just bounded by the amount of space avalible to the system. This is also extremely inefficient example of memory management, which could impact the speed at which the code executes. One other implementation could be an array which doubles in sizes every time it becomes filled, copying all elements into the doubled ever into the new array, and storing the size of array. The allocated memory would have to be updated at the end of the code execution, shifting each element in the stack for addressing.

### Self Reflection

#### Quinn
##### How much did you know about the subject before we started?
I had some prior knowledge about coding in assembly from past courses, however have not manually translated python code into assembly. Pep/9 proved to be a challenge, as not only was the syntax different, but I my depth of knowledge did not extend past conditionals.

##### What did/do you find frustrating about this assignment?
The entire process was frustrating, from tackling a design pattern I have never seen before, to wresting with learning assembly. In addition, navigating the ast library was difficult and frustrating, however proved useful in the overal code implementation. 

I would have wished to have more time to learn about the codebase, and tools availible at hand in order to produce code of higher quality. This includes code that follows the SOLID principles, as well as easily traceable code. I was frustrated with not being able to produce code that I would have been proud of due to time and knowledge constraints.

##### If you were the instructor, what comments would you make about this piece?
I would comment that while the code functions, it may be difficult to read/understand, or even to follow. I would recommend to review design patterns, and look to see how we could implement some of them to better structure and organize the code.

##### What would you change if you had a chance to do this assignment over again?
I would first establish a common baseline of knowledge between me and my groupmate, as we both understood various components of the task in various different manners. For example, it took me a long time to understand how the stack pointer functioned in assembly, and we ended up having different code/knowledge and expectations which did not merge well. In addition, I would produce more robust code for handling functions, as some test cases do indeed fail.


#### Fondson
##### How much did you know about the subject before we started?
Before the lab began, I had a very low amount of knowledge regarding assembly language. I have also never used the Pep/9 application in any sort of previous lab or project. At the start of the semester, we were told to download Pep/9 as it’ll be required later into the course (now). I’ve personally never looked into assembly language nor the application of Pep/9 until the lab began. This is by far my worst regret as I should’ve come more prepared entering the project. 

##### What did/do you find frustrating about this assignment?
Throughout this project, the most frustrating task would be attempting to improve the translator to address three limitations in the translation process of F2. I personally struggled to understand how to improve on the constants regarding the F2 task. It was later figured out that in order to improve on the constants, we implemented the idea to store the old variable name and further pass it into a generator to detect if it was a constant. This returned a keyword to help derive whether it was a constant or not.

##### If you were the instructor, what comments would you make about this piece?
If I were the instructor marking this lab, I think a good comment I would make regarding this piece would be the well detailed explanation of each task throughout this lab. We have incrementally submitted reports throughout the lab weeks and have carefully explained our thought process and design/solutions throughout this project. The capability of explaining the thought process behind work is essential in a well performed project.

##### What would you change if you had a chance to do this assignment over again?
If I had the chance to do this assignment over again, I think the best approach would be to play a more active role throughout the development of the code. Throughout the time spent on this lab, my lab partner was very helpful. The real challenge and learning experience would be to put myself through struggle, and redo the entire lab, to ensure that I’ve attained the proper knowledge received from this project.